## Combining Datasets (Concat, Append, Merge & Join)

These operations can involve anything from very straightforward concatenation of two different datasets, to more complicated database-style joins and merges that correctly handle any overlaps between the datasets. Series and DataFrames are built with this type of operation in mind, and Pandas includes functions and methods that make this sort of data wrangling fast and straightforward. Here we’ll take a look at simple concatenation of Series and DataFrames with the pd.concat function; later we’ll dive into more sophisticated in-memory merges and joins implemented in Pandas.



In [1]:
import pandas as pd
import numpy as np

In [2]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c:[str(c) + str(i) for i in ind] for c in cols}
    print(data)
    return pd.DataFrame(data, ind)

In [3]:
make_df.__doc__

'Quickly make a DataFrame'

In [4]:
make_df('ABC', range(3))

{'A': ['A0', 'A1', 'A2'], 'B': ['B0', 'B1', 'B2'], 'C': ['C0', 'C1', 'C2']}


,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


#### Quick Concat Reminder (Numpy)

In [9]:
x = [1,2,3]
y = [4,5,6]
z = [7,8,9]
[np.concatenate([x, y, z]), type(np.concatenate([x, y, z])), np.concatenate([x, y, z]).shape]

[array([1, 2, 3, 4, 5, 6, 7, 8, 9]), numpy.ndarray, (9,)]

In [11]:
# Speciy axis along with the result will be concatenated
x_1 = [[1,2,3], [4,5,6]]
np.concatenate([x_1, x_1], axis=1)

array([[1, 2, 3, 1, 2, 3],
       [4, 5, 6, 4, 5, 6]])

In [12]:
np.concatenate([x_1, x_1], axis=0)

array([[1, 2, 3],
       [4, 5, 6],
       [1, 2, 3],
       [4, 5, 6]])

AxisError: axis 1 is out of bounds for array of dimension 1

#### Pandas Concat

``` python
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True) pd.concat()
```
* pd.concat() be used for a simple concatenation of Series or DataFrame objects, just as np.concatenate() can be used for simple concatenations of arrays:



In [18]:
# Note here even a shared index value will have two row outputs
ser_1 = pd.Series(['A', 'B', 'C'], index=[1,2,3])
ser_2 = pd.Series(['D', 'E', 'F'], index=[4,3,6])
pd.concat([ser_1, ser_2])

1    A
2    B
3    C
4    D
3    E
6    F
dtype: object

In [20]:
df_1 = make_df('AB', [1,2])
df_2 = make_df('AB', [3,4])
pd.concat([df_1, df_2])

{'A': ['A1', 'A2'], 'B': ['B1', 'B2']}
{'A': ['A3', 'A4'], 'B': ['B3', 'B4']}


,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


* By default, the concatenation takes place row-wise within the DataFrame (i.e., axis=0). Like np.concatenate, pd.concat allows specification of an axis along which concatenation will take place.
* The default is stacking in a long (vertical) fashion whereas use of (axis=1) (see df_3 and df_4 below) stacks in a wide format

In [15]:
df_3 = make_df('AB', [0, 1])
df_4 = make_df('CD', [0,1])
display(df_3)
display(df_4)

{'A': ['A0', 'A1'], 'B': ['B0', 'B1']}
{'C': ['C0', 'C1'], 'D': ['D0', 'D1']}


,A,B
0,A0,B0
1,A1,B1


,C,D
0,C0,D0
1,C1,D1


In [16]:
print(pd.concat([df_3, df_4], axis=1))

    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


In [26]:
print(pd.concat([df_3, df_4])) 
# default different columns when stacked bring in all data and simply represent NAN for values 
# of the other dataframe not having the column

     A    B    C    D
0   A0   B0  NaN  NaN
1   A1   B1  NaN  NaN
0  NaN  NaN   C0   D0
1  NaN  NaN   C1   D1


### Duplicate indices 
* One important difference between np.concatenate and pd.concat is that Pandas concatenation preserves indices, even if the result will have duplicate indices! Consider this simple example:

In [17]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2,3])
display(x)
display(y)
print(x.index, y.index)

{'A': ['A0', 'A1'], 'B': ['B0', 'B1']}
{'A': ['A2', 'A3'], 'B': ['B2', 'B3']}


,A,B
0,A0,B0
1,A1,B1


,A,B
2,A2,B2
3,A3,B3


Int64Index([0, 1], dtype='int64') Int64Index([2, 3], dtype='int64')


In [34]:
print(pd.concat([x, y]))

    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


In [20]:
y.index = x.index
print(pd.concat([x, y]))

    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


* Although valid, the outcome of multiple indices of the same value is often undesirable. Let's see how we can handle

#### Catching the repeats as an error
* To simply verify the indices in the result of pd.concat() do not overlap, the `verify_integrity` flag can be specified. When set to `True`, the concatenation will raise an exceiption if there are duplicate indicies.

In [21]:
try:
    pd.concat([x, y], verify_integrity=True)
except Exception as e:
    print(type(e))
    print(e)

<class 'ValueError'>
Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


In [22]:
# Can Ignore the Index
pd.concat([x,y], verify_integrity=False)

,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


#### Adding MultiIndex Keys
The `keys` argument to specify a label for the data sources; result with be hierarchically indexed series containing the data

In [23]:
display(x)
display(y)

,A,B
0,A0,B0
1,A1,B1


,A,B
0,A2,B2
1,A3,B3


In [24]:
print(pd.concat([x, y], keys=['x', 'y']))

      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3


In [25]:
z_midx_keys = pd.concat([x,y], keys=['x', 'y'])
z_midx_keys

A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

#### Quick Reminder on Indexing Hierarchical Levels

In [28]:
z_midx_keys.loc[[('x', 1), ('y', 0)], 'B']

x  1    B1
y  0    B2
Name: B, dtype: object

### Concatenation w/Joins
In practice, data from different sources might have different sets of column names, and pd.concat offers several options. 
* Let's start with combining two dataframes that have some (not all) columns in common

In [44]:
df_5 = make_df('ABC', [1,2])
df_6 = make_df('BCD', [3,4])
pd.concat([df_5, df_6], keys=['df_5', 'df_6'])

{'A': ['A1', 'A2'], 'B': ['B1', 'B2'], 'C': ['C1', 'C2']}
{'B': ['B3', 'B4'], 'C': ['C3', 'C4'], 'D': ['D3', 'D4']}


A   B   C    D
df_5 1   A1  B1  C1  NaN
     2   A2  B2  C2  NaN
df_6 3  NaN  B3  C3   D3
     4  NaN  B4  C4   D4

* By default, the entries for which no data is available (example column D for key df_5 (or that dataframe) is filled by default with NA values. 
* Using the `join` argument we can specify what we'd like to concatenate
    * By default this uses the `join='outer'` and thus returns all values from the attempted join

In [45]:
print(pd.concat([df_5, df_6], join='inner'))

    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4


* Columns not present in either df_5 or df_6 ('A' & 'D') are not included

In [48]:
pd.concat([df_5, df_6], axis=1)

,A,B,C,B,C,D
1,A1,B1,C1,NaN,NaN,NaN
2,A2,B2,C2,NaN,NaN,NaN
3,NaN,NaN,NaN,B3,C3,D3
4,NaN,NaN,NaN,B4,C4,D4


In [52]:
df1_axis = pd.DataFrame([['a', 1], ['b', 2]], columns=['letter', 'number'])
df4_axis = pd.DataFrame([['bird', 'polly'], ['monkey', 'george']], columns=['animal', 'name'])
display(df1_axis)
display(df4_axis)
display(pd.concat([df1_axis, df4_axis], axis=1))
display(pd.concat([df1_axis, df4_axis], axis=0))

,letter,number
0,a,1
1,b,2


,animal,name
0,bird,polly
1,monkey,george


,letter,number,animal,name
0,a,1,bird,polly
1,b,2,monkey,george


,letter,number,animal,name
0,a,1.0,NaN,NaN
1,b,2.0,NaN,NaN
0,NaN,NaN,bird,polly
1,NaN,NaN,monkey,george


### The append() method
* pd.concat([df_1, df_2]) can simple also be df_1.append(df_2)

In [53]:
df_1.append(df_2)

/var/folders/r8/mmfd5rgj7v51_80zhnq8v9zc0000gn/T/ipykernel_33048/2678984180.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1.append(df_2)


,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


* However, concat as we can see should be used with the FutureWarning above for future versions

### Merge & Join
* In-memory join and merge operations
    * Very similar to RDBMS (Database) style merges/joins

#### One-to-one joins

In [29]:
dframe_1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'], 'group': ['Accounting', 'Engineering', 
                                                                               'Engineering', 'HR']})
dframe_2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'], 'hire_data': [2004, 2008, 2012, 2014]})

In [30]:
dframe_1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [31]:
dframe_2

,employee,hire_data
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [32]:
dframe_3 = pd.merge(dframe_1, dframe_2)
dframe_3

,employee,group,hire_data
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


* The pd.merge() function recognizes that each DataFrame has an “employee” column, and automatically joins using this column as a key. The result of the merge is a new DataFrame that combines the information from the two inputs. Notice that the order of entries in each column is not necessarily maintained: in this case, the order of the “employee” column differs between df1 and df2, and the pd.merge() function correctly accounts for this.

#### Many-to-one joins
* Joins in which one of the two key columns contains duplicate entries.
* Resulting DataFrame will preserve those duplicate entries as appropriate

In [58]:
dframe_4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'], 
                        'supervisor': ['Carly', 'Guido', 'Steve']})
dframe_4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [59]:
print(pd.merge(dframe_3, dframe_4))

  employee        group  hire_data supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve


* The resulting DataFrame has an additional column with the “supervisor” information, where the information is repeated in one or more locations as required by the inputs.
* Enginnering employees are coupled with one manager so the value in dframe_4 for (same index in each column) is assigned to any Engineering employee

#### Many-to-many joins

In [33]:
dframe_5 = pd.DataFrame({'group':['Accounting', 'Accounting', 'Engineering',
                                 'Engineering', 'HR', 'HR'],
                        'skills':['math', 'spreadsheets', 'coding', 'linux',
                                 'spreadsheets', 'organization']})
print(pd.merge(dframe_1, dframe_5))

  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


#### Quick Notes
* Engineering duplicate values in group allocates the two skills at the matching column level in above dataframe to attribute the same skills column index to the individuals existing in dframe_1 for the merged group column

### Specificaiton of the Merge Key
* Above is the default behavior for pd.merge()
    * It looks for one or more matching columns names between the two inputs, and uses this as the key
* However, often the column names will not match so nicely
    * As such, pd.merge() provides a variety of options for handling

#### On Keyword

In [34]:
display(dframe_1)
display(dframe_2)

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


,employee,hire_data
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [35]:
print(pd.merge(dframe_1, dframe_2, on='employee'))

  employee        group  hire_data
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


* Option only works if both the left and right `DataFrames` have the specified column name as the example above shows

#### The left_on and right_on keywords
* At times you may wish to merge two datasets with different column names; for example, we may have a dataset in which the employee name is labeled as `“name”` rather than `“employee”`. 
* In this case, we can use the `left_on` and `right_on` keywords to specify the two column names


In [36]:
dframe_3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                        'salary': [70000, 80000, 120000, 90000]})
dframe_3

,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [37]:
dframe_1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [38]:
pd.merge(dframe_1, dframe_3, left_on='employee', right_on='name')

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


In [39]:
pd.merge(dframe_3, dframe_1, left_on='name', right_on='employee')

,name,salary,employee,group
0,Bob,70000,Bob,Accounting
1,Jake,80000,Jake,Engineering
2,Lisa,120000,Lisa,Engineering
3,Sue,90000,Sue,HR


In [40]:
# drop redundant column joined with different names 
# (drop defaults to rows so use non default (axis=1) to drop redundant column)
pd.merge(dframe_1, dframe_3, left_on='employee', right_on='name').drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### The left_index and right_index keywords
* Sometimes, rather than merging on a column, you would instead like to merge on an index

In [41]:
df1a = dframe_1.set_index('employee')
df1a

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [42]:
df2a = dframe_2.set_index('employee')
df2a

,hire_data
employee,
Lisa,2004
Bob,2008
Jake,2012
Sue,2014


In [43]:
pd.merge(df1a, df2a, left_index=True, right_index=True)

,group,hire_data
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


* `DataFrames` implement the join() method, which perorms a merge that defaults to joining on indices

In [44]:
df1a.join(df2a)

,group,hire_data
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


* To mix indices and columns, combine left_index with right_on or left_on with right_index 

In [45]:
display(df1a)
display(dframe_3)
pd.merge(df1a, dframe_3, left_index=True, right_on='name')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


,group,name,salary
0,Accounting,Bob,70000
1,Engineering,Jake,80000
2,Engineering,Lisa,120000
3,HR,Sue,90000


### Specifying Set Arithmetic for Joins

* Above we have glossed over one important consideration in performing a join:
    * The type of set arithmetic used in the join
    * This occurs when a  value appears in one key column but not the other

In [49]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                   'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                   'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])

[display(x) for x in [df6, df7]]

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


,name,drink
0,Mary,wine
1,Joseph,beer


[None, None]

In [51]:
pd.merge(df6, df7) # should look at shared column name (Only Mary in both dataframes above)

,name,food,drink
0,Mary,bread,wine


In [52]:
pd.merge(df6, df7, how='inner') # same as default above (I would think)

,name,food,drink
0,Mary,bread,wine


* `how` keyword options like 'outer', 'left', 'right' also avaialble (just like a RDBMS)
    * Outer join returns a join over the union of the input columns, and fills in all missing values with NAs

In [54]:
pd.merge(df6, df7, how='outer') # Mary is the only name in both and thus only row without Nulls

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


In [56]:
pd.merge(df6, df7, how='left') # take all values from left table and match if value found in df7 otherwise don't return

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


In [57]:
pd.merge(df6, df7, how='right') # Inverse of above but generally just left joins like SQL with table designation of left/right

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


### Overlapping Column Names (suffixes Keyword)
* You may end up in a case where your two input `DataFrames` have conflicting column names. 

In [58]:
df8 = pd.DataFrame({'name':['Bob', 'Jake', 'Lisa', 'Sue'],
                   'rank':[1, 2, 3, 4]})
df9 = pd.DataFrame({'name':['Bob', 'Jake', 'Lisa', 'Sue'],
                   'rank':[3,1,4,2]})

[display(x) for x in [df8, df9]]

,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


,name,rank
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


[None, None]

In [59]:
pd.merge(df8, df9, on='name')

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


* Because the output would have two conflicting column names, the merge function automatically appends a suffix_x or _y to make the ouptut columns unique.

In [60]:
pd.merge(df8, df9, on='name', suffixes=['_L', '_R'])

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


### Example: US States Data
* Can run following shell commands in Jupyter environment to import csv datasets

In [63]:
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57935  100 57935    0     0   574k      0 --:--:-- --:--:-- --:--:--  577k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   835  100   835    0     0   4426      0 --:--:-- --:--:-- --:--:--  4441
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   872  100   872    0     0   3849      0 --:--:-- --:--:-- --:--:--  3841


In [64]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')
[display(x.head()) for x in [pop, areas, abbrevs]]

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


[None, None, None]

* We want to rank US states and territories by their 2010 population density. To do such we'll have to combine the datasets to get it
* We can merge based on the state/region column of pop, and the abbreviation column of abbrevs

In [69]:
pop_abbrevs_inner = pd.merge(pop, abbrevs, left_on='state/region', right_on='abbreviation')
pop_abbrevs_outer = pd.merge(pop, abbrevs, how='outer', left_on='state/region', right_on='abbreviation')
print(len(pop_abbrevs_inner), len(pop_abbrevs_outer), '\n',
      pop_abbrevs_inner.isna().sum(), '\n', pop_abbrevs_outer.isna().sum())

2448 2544 
 state/region    0
ages            0
year            0
population      0
state           0
abbreviation    0
dtype: int64 
 state/region     0
ages             0
year             0
population      20
state           96
abbreviation    96
dtype: int64


* The union ('outer') returns about 100 more rows which would suggest we have some nulls or abbrevs in one dataframe as opposed to the default which is looking for solely the intersection and not a 'union' like 'outer'
* Outer will make sure no data is thrown away due to mismatched labels

In [70]:
pop_abbrevs_outer.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL


In [71]:
# drop duplicate abbreviation 
pop_abbrevs_outer.drop('abbreviation', axis=1, inplace=True)
pop_abbrevs_outer.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [72]:
# Null shown above but can look for a simple boolean null check on the column too
pop_abbrevs_outer.isna().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [73]:
# What's Null?
pop_abbrevs_outer.loc[pop_abbrevs_outer['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [74]:
# Just looks like the state/region with null population values is Puerto Rico
pop_abbrevs_outer.loc[pop_abbrevs_outer['population'].isnull()]['state/region'].unique()

array(['PR'], dtype=object)

* Null population appears to be from Puerto Rico
* Some of the state entries are also null, there was no corresponding entry in the abbrevs key

In [76]:
# Locate null values/row for the state column and find unique state/region values for null values in state column
pop_abbrevs_outer.loc[pop_abbrevs_outer['state'].isnull()]['state/region'].unique()

array(['PR', 'USA'], dtype=object)

* our population data includes entries for Puerto Rico (PR) and the United States (US) as  a whole, while these entries do not appear in the state abbreviation key from the abbrevs dataframe
* Can fill in appropriate entries

In [77]:
# locate PR/USA state/region values and isolate state columns for found conditional then set 'state'
pop_abbrevs_outer.loc[pop_abbrevs_outer['state/region'] == 'PR', 'state'] = 'Puerto Rico'
pop_abbrevs_outer.loc[pop_abbrevs_outer['state/region'] == 'USA', 'state'] = 'United States'
# See that null from 'state' is no more
pop_abbrevs_outer.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [78]:
# Now include Areas (share same column to merge on)
# left join to include all from first merged dataframe and matched states found for the areas dataframe
all_three = pd.merge(pop_abbrevs_outer, areas, on='state', how='left')
all_three.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [79]:
all_three.isna().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [80]:
all_three.isna().sum()

state/region      0
ages              0
year              0
population       20
state             0
area (sq. mi)    48
dtype: int64

In [88]:
# Make sure for double bracket notation when subsetting subsequent columns to look at when evaluating null columns value across other columns
all_three.loc[all_three['area (sq. mi)'].isna()][['state/region', 'year', 'population']].head(10)

,state/region,year,population
2496,USA,1990,64218512.0
2497,USA,1990,249622814.0
2498,USA,1991,252980942.0
2499,USA,1991,65313018.0
2500,USA,1992,66509177.0
2501,USA,1992,256514231.0
2502,USA,1993,259918595.0
2503,USA,1993,67594938.0
2504,USA,1994,68640936.0
2505,USA,1994,263125826.0


In [89]:
all_three.loc[all_three['area (sq. mi)'].isna()]['state/region'].unique()

array(['USA'], dtype=object)

In [92]:
[all_three['state/region'].nunique(), [all_three['state/region'].unique()]]

[53,
 [array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
         'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
         'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
         'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
         'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR', 'USA'],
        dtype=object)]]

* Nulls in the area column all have a state/region of 'USA'
* This suggests the DataFrame does not contain the area of the United States as a whole as the state/region value for null area values only attributes to this state/region and not any of the others we can see above

In [93]:
# The population density of the entire US (sample rows for years above) is not relevant to our current path so we can just drop inplace
all_three.dropna(inplace=True)
all_three.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


* We have all the data we need now
* We can initially select the portion of the data corresponding with the year 2010 & the total population
* the query() functon can perform this quickly

In [100]:
data_2010 = all_three.query("year == 2010 & ages == 'total'")
data_2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [101]:
data_2010['year'].unique()

array([2010])

In [102]:
# Compute population density and display it in order
# Reindex our data on the state

data_2010.set_index('state', inplace=True)
density = data_2010['population'] / data_2010['area (sq. mi)']
density.sort_values(ascending=False, inplace=True)
print(type(density))
density.head()

<class 'pandas.core.series.Series'>


state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

* Result is a ranking of US states plus Washington DC & Puerto Rico in order of their 2010 population density in residents per square mile. No surprise that the densest region in this dataset is Washington DC

In [103]:
# What about our least dense
density.tail(6)

state
New Mexico      16.982737
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64